### Data Augmentation

We need to have our folder with the base training and another folder where we will store the transormed images

In [ ]:
import os
import cv2 as cv
from tqdm import tqdm
from helpers import *

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

In [ ]:
BASE_TRAINING = './base_training/'
BASE_TRAIN_IMAGES = BASE_TRAINING + 'images/'
BASE_TRAIN_GROUNDTRUTH = BASE_TRAINING + 'groundtruth/'

TRAINING = './training/'
TRAIN_IMAGES = TRAINING + 'images/'
TRAIN_GROUNDTRUTH = TRAINING + 'groundtruth/'

### Add 90, 180 and 270º rotations

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_rotations = get_rotations_0_90_180_270(image)
    for i, rotated_image in enumerate(img_train_rotations[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_rotation_' + str((i+1)*90) + ".png"), rotated_image)

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    img_train_rotations = get_rotations_0_90_180_270(image)
    for i, rotated_image in enumerate(img_train_rotations[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_rotation_' + str((i+1)*90) + ".png"), rotated_image)

### Add randomly rotated images

In [ ]:
ROTATIONS_PER_IMAGE = 3
degrees = []
for i in range(ROTATIONS_PER_IMAGE*len(os.listdir(BASE_TRAIN_IMAGES))):
    degrees.append(random.randint(0, 90))

In [ ]:
index = 0
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    for i in range(ROTATIONS_PER_IMAGE):
        random_rotation = get_rotation_deg_n(image, degrees[index])
        cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_random_rotation_' + str(i) + ".png"), random_rotation)
        index += 1

In [ ]:
index = 0
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    for i in range(ROTATIONS_PER_IMAGE):
        random_rotation = get_rotation_deg_n(image, degrees[index])
        cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_random_rotation_' + str(i) + ".png"), random_rotation)
        index += 1

### Add horizontally and vertically flipped images

In [ ]:
flip_type = ['x', 'y'] # Removed x-y rotation as it is equivalent to 180 rotation

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_flips = get_flipped_images(image)
    for i, flipped_image in enumerate(img_train_flips[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_flipped_' + flip_type[i] + ".png"), flipped_image)

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    img_train_flips = get_flipped_images(image)
    for i, flipped_image in enumerate(img_train_flips[1:]): # Avoid original image
        cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_flipped_' + flip_type[i] + ".png"), flipped_image)

### Add gaussian noise

In [ ]:
%run helpers.py
VARIANCE = 200

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_noise = noisy('gauss', image, var=VARIANCE)
    cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_noise_' + 'gauss_var_' + str(VARIANCE) + ".png"), img_train_noise)

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_noise_' + 'gauss_var_' + str(VARIANCE) + ".png"), image)

### Add salt and pepper noise

In [ ]:
%run helpers.py
CORRUPTION_RATIO = 0.01875

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    img_train_noise = noisy('s&p', image, var=VARIANCE)
    cv.imwrite(os.path.join(TRAIN_IMAGES , img_name[:-4] + '_noise_' + 's&p_corrupt_' + str(CORRUPTION_RATIO) + ".png"), img_train_noise)

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH , img_name[:-4] + '_noise_' + 's&p_corrupt_' + str(CORRUPTION_RATIO) + ".png"), image)

### Add original images

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_IMAGES), total=len(os.listdir(BASE_TRAIN_IMAGES))):
    image = cv.imread(BASE_TRAIN_IMAGES + img_name)
    cv.imwrite(os.path.join(TRAIN_IMAGES + img_name), image)

In [ ]:
for img_name in tqdm(os.listdir(BASE_TRAIN_GROUNDTRUTH), total=len(os.listdir(BASE_TRAIN_GROUNDTRUTH))):
    image = cv.imread(BASE_TRAIN_GROUNDTRUTH + img_name)
    image = image[:, :, 0]
    cv.imwrite(os.path.join(TRAIN_GROUNDTRUTH + img_name), image)

### Delete images

In [ ]:
for filename in tqdm(os.listdir(TRAIN_IMAGES),total=len(os.listdir(TRAIN_IMAGES))):
    file_path = os.path.join(TRAIN_IMAGES, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))
        
for filename in tqdm(os.listdir(TRAIN_GROUNDTRUTH),total=len(os.listdir(TRAIN_GROUNDTRUTH))):
    file_path = os.path.join(TRAIN_GROUNDTRUTH, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))